In [1]:
import numpy as np
import pandas as pd

In [2]:
import os 
os.chdir("../")

#### 1.数据导入

In [3]:
# 净值、价格和成交量数据
lof_open = pd.read_csv("./Data/LOF开盘价.CSV",index_col = 0)
lof_close = pd.read_csv("./Data/LOF收盘价.CSV",index_col = 0)
lof_high = pd.read_csv("./Data/LOF最高价.CSV",index_col = 0)
lof_low = pd.read_csv("./Data/LOF最低价.CSV",index_col = 0)
lof_nav = pd.read_csv("./Data/LOF复权净值.CSV",index_col = 0)
lof_volume = pd.read_csv("./Data/LOF成交量.CSV",index_col = 0)

In [4]:
# 将SZ,SH结尾的场内LOF转换成OF结尾的
lof_open.columns = lof_open.columns.map(lambda x:x[:6]+".OF")
lof_close.columns = lof_close.columns.map(lambda x:x[:6]+".OF")
lof_high.columns = lof_high.columns.map(lambda x:x[:6]+".OF")
lof_low.columns = lof_low.columns.map(lambda x:x[:6]+".OF")
lof_nav.columns = lof_nav.columns.map(lambda x:x[:6]+".OF")
lof_volume.columns = lof_volume.columns.map(lambda x:x[:6]+".OF")

In [5]:
lof_open.index = pd.to_datetime(lof_open.index)
lof_close.index = pd.to_datetime(lof_close.index)
lof_high.index = pd.to_datetime(lof_high.index)
lof_low.index = pd.to_datetime(lof_low.index)
lof_nav.index = pd.to_datetime(lof_nav.index)
lof_volume.index = pd.to_datetime(lof_volume.index)

In [6]:
# 导入到期日数据
maturity_period = pd.read_excel("./Output/MaturePeriod.xlsx",index_col = 0)

In [7]:
# 导入申购状态数据
redemption_status = pd.read_excel("./Data/RedemptionStatus.xlsx",index_col = 0)
subscribe_status = pd.read_excel("./Data/SubscribeStatus.xlsx",index_col = 0)

In [8]:
redemption_status.drop("F_INFO_WINDNAME",axis = 1,inplace = True)
subscribe_status.drop("F_INFO_WINDNAME",axis = 1,inplace = True)

#### 2.获取300个交易日的相对底部

In [9]:
start_date = pd.to_datetime("2014-01-01")
end_date = pd.to_datetime("2022-03-29")

target_percentage = 15
backward_days = 150

In [10]:
bottom_info = pd.DataFrame(index = redemption_status.index,columns = redemption_status.loc[:,start_date:end_date].columns,data = False)

In [11]:
for tmp_date in lof_close.loc[start_date:,:].index: # 是购买日
    for tmp_lof in lof_close.columns:
        price_length = len(lof_close.loc[:tmp_date,tmp_lof].dropna().iloc[-backward_days:])
        if price_length < backward_days:
            continue
#         print(lof_close.loc[:tmp_date,tmp_lof].dropna().iloc[-151:-1]) # 除去当前交易日的前150个有价格的数据
        tmp_price = lof_close.loc[:tmp_date,tmp_lof].dropna().iloc[-1] # 当前交易日前一个交易日的收盘价
        tmp_target_pct_value = np.percentile(lof_close.loc[:tmp_date,tmp_lof].dropna().iloc[-backward_days:],target_percentage)
        if tmp_price <= tmp_target_pct_value:
            bottom_info.loc[tmp_lof,tmp_date] = True

In [12]:
bottom_info.to_excel("./Output/BottomInfo.xlsx")